In [ ]:
!pip install transformers

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
os.chdir('drive/My Drive/project_path')

In [ ]:
import torch
from transformers import AutoModel, AutoTokenizer
import logging
import matplotlib.pyplot as plt
from os import path

In [ ]:
def get_bert_input(text, tokenizer):
  # text = "프로그래머"
  marked_text = "[CLS]" + text + "[SEP]"
  tokenized_text = tokenizer.tokenize(marked_text)
  indexed_tokens = tokenizer.convert_tokens_to_ids(tokenized_text)
  # print(tokenized_text)
  # print(indexed_text)
  segments_ids = [1]
  tokens_tensor = torch.tensor([indexed_tokens])
  segments_tensor = torch.tensor([segments_ids])
  return tokens_tensor, segments_tensor

In [ ]:
def get_word_emb(text, tokenizer, model):
  tokens_tensor, segments_tensor = get_bert_input(text, tokenizer)
  model.eval()
  with torch.no_grad():
    outputs = model(tokens_tensor, segments_tensor)
  hidden_states = outputs[2]
  token_vecs = hidden_states[-2][0] # second to last layer representation. (num_tokens, 768)
  # Calculate the average of all 22 token vectors.
  sentence_embedding = torch.mean(token_vecs, dim=0)
  return sentence_embedding

In [ ]:
model = AutoModel.from_pretrained("klue/bert-base", output_hidden_states = True)
tokenizer = AutoTokenizer.from_pretrained("klue/bert-base")

Downloading:   0%|          | 0.00/425 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/424M [00:00<?, ?B/s]

Some weights of the model checkpoint at klue/bert-base were not used when initializing BertModel: ['cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.decoder.bias', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Downloading:   0%|          | 0.00/289 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/243k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/483k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/125 [00:00<?, ?B/s]

In [ ]:
emb1 = get_word_emb("프로그래머", tokenizer, model)

In [ ]:
gn_dict = {}
import csv
f = open('./word list/professions.txt', 'r')    # change the path : './word list/male.txt', './word list/female.txt', './word list/gender_neutral.txt', or './word list/professions.txt'
while True:
    line = f.readline()
    line = line.strip()
    if not line: break
    if line == '': continue
    gn_dict[line] = get_word_emb(line, tokenizer, model).tolist()
    # print(line)
f.close()

In [ ]:
import json
with open('./word list/professions.json','w') as f: # change the path : './word list/male.json', './word list/female.json', './word list/gender_neutral.json', or './word list/professions.json'
    json.dump(gn_dict,f)